In [1]:
import os
import tqdm
import gensim
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn import preprocessing
from geopandas.tools import sjoin
from gensim.models.doc2vec import TaggedDocument

## Settings

In [2]:
# Dir
data_dir = './SZ_data/'
# Seeds
seed = 1
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
# Flags
poi_exist = True
unit_exist = True
area_exist = True
seq_exist = True
model_exist = True
flow_exist = True
clean_exist = True
dis_exist = True
adj_exist = True

## POIs (crs:epsg - 4326)

In [3]:
def encode_poitype(df):
    df_sl = df.copy()
    df_sl[["s1_1", "s1_2", "s1_3", "s1_4", "s1_5", "s1_6"]] = pd.DataFrame(df_sl["type"].str.split("|").to_list())
    df_sl[["level_1", "level_2", "level_3"]] = pd.DataFrame(df_sl["s1_1"].str.split(";").to_list(), index=df_sl.index)
    df_sl = df_sl[["wgslng", "wgslat", "level_1", "level_2"]].sort_values("level_1")[2:].reset_index(drop=True)
    return df_sl

In [4]:
# GeoDataFrame 1,577,354 * (wgslng, wgslat, level_1, level_2, code, geometry)
if not poi_exist:
    p_poi = data_dir + 'org_data/SZ_poi_2018.csv'
    df_poi = pd.read_csv(p_poi,encoding="utf-8")
    df_poi_c = encode_poitype(df_poi)
    df_poi_c = df_poi_c[~df_poi_c.duplicated(keep="first")]
    le_poi_first_level = preprocessing.LabelEncoder()
    le_poi_second_level = preprocessing.LabelEncoder()
    df_poi_c['level_1'] = le_poi_first_level.fit_transform(df_poi_c['level_1'].values)
    df_poi_c['level_2'] = le_poi_second_level.fit_transform(df_poi_c['level_2'].values)
    df_poi_c['code'] = df_poi_c.apply(lambda x: f"{x['level_1']:.0f}{x['level_2']:.0f}", axis=1)
    gdf_poi = gpd.GeoDataFrame(df_poi_c,geometry=gpd.points_from_xy(df_poi_c['wgslng'],df_poi_c['wgslat'],crs='epsg:4326'))
    gdf_poi.to_file(data_dir+'intermediate_results/poi.shp',encoding = 'utf-8')
else:
    gdf_poi = gpd.read_file(data_dir+'intermediate_results/poi.shp')

## Spatial units (crs:epsg - 32650)

In [5]:
# GeoDataFrame 33,054 * (ID, geometry)
if not unit_exist:
    p_unit = data_dir + 'org_data/LU_250.shp'
    gdf_unit = gpd.read_file(p_unit)
    gdf_unit = gdf_unit[~gdf_unit.duplicated(subset=['FID_'],keep="first")]
    gdf_unit = gdf_unit[['FID_','geometry']].rename(columns={"FID_":"ID"})
    gdf_unit.to_file(data_dir+'intermediate_results/unit.shp',encoding='utf-8')
else:
    gdf_unit = gpd.read_file(data_dir+'intermediate_results/unit.shp')

## Land use

In [6]:
def get_classremap_dic(src_path,des_path):
    df_class = pd.read_excel(src_path,skiprows=2,names = [str(i) for i in range(6)])
    df_class["5"] = df_class["5"].fillna(method="ffill")
    df_class = df_class.dropna(subset=["4"])
    class_mapdict = {x["4"]: x["5"] for _, x in df_class[df_class["4"] != "/"][["4", "5"]].iterrows()}
    class_mapdict["特殊用地"] = "绿地广场用地"
    class_mapdict["水工建筑用地"] = "公用设施用地"
    with open(des_path,'wb') as f:
        pickle.dump(class_mapdict, f)
    return class_mapdict

In [7]:
def pivot_area(df):
    df_area = df.copy()
    p_class = data_dir + 'org_data/三调用地分类_更新.xlsx'
    des_class = data_dir + 'org_data/class_mapdict.pkl'
    class_mapdict = get_classremap_dic(src_path = p_class, des_path = des_class)
    df_area["CLA"] = df_area["DLMC"].apply(lambda x: class_mapdict[x])
    df_area = df_area.groupby(["ID", "CLA"]).sum("PERCENTAGE").reset_index()
    df_area = df_area.pivot(index="ID", columns="CLA",values="PERCENTAGE").fillna(0)
    return df_area

In [8]:
# DataFrame 33,054 * (ID, 公共管理与公共服务用地, 公用设施用地, 商业服务设施用地, 居住用地, 工业用地, 绿地广场用地, 道路与交通设施用地, 非建设用地)
if not area_exist:
    p_lu = data_dir + 'org_data/area_250m_fn.txt'
    df_lu = pd.read_csv(p_lu)
    df_lu = df_lu.drop(columns=["OID_"]).rename(columns={"FID_":"ID"})
    df_area = pivot_area(df_lu)
    df_area = (df_area/100).round(3)
    df_area["非建设用地"] = df_area.apply(lambda x: (1-x.drop("非建设用地").sum()), axis=1)
    df_area.to_csv(data_dir+'intermediate_results/unit_area.csv')
else:
    df_area = pd.read_csv(data_dir+'intermediate_results/unit_area.csv')

## Generate sequences & Embedding

In [9]:
def get_sequences_by_distancegreedy(gdf_tz, gdf_poi, minpois = 10):
    sequences = {}
    df_join = sjoin(gdf_poi, gdf_tz, how="inner",op="within")
    for tz_ind in tqdm.tqdm(df_join.index_right.unique()):
        tz_pois = df_join[df_join.index_right == tz_ind].reset_index()
        if tz_pois.shape[0] > minpois:
            pnt_num = tz_pois.shape[0]
            z = np.array([[complex(g.x, g.y) for g in tz_pois.geometry]])
            dismat = abs(z.T-z)
            visited = list(np.unravel_index(np.argmax(dismat, axis=None), dismat.shape))
            ## list of to be visited points
            not_visited = [x for x in range(pnt_num) if x not in visited]
            np.random.shuffle(not_visited)
            while not_visited:
                to_be_visit = not_visited.pop()
                if len(visited) == 2:
                    visited.insert(1, to_be_visit)
                    pass
                else:
                    search_bound = list(zip(visited[0:-1], visited[1:]))
                    dis = [dismat[to_be_visit, x]+dismat[to_be_visit, y] for x, y in search_bound]
                    insert_place = dis.index(min(dis))+1
                    visited.insert(insert_place, to_be_visit)
            sequences[tz_ind] = tz_pois.loc[visited, "code"].values
    return sequences

In [10]:
if not seq_exist:
    sequences = get_sequences_by_distancegreedy(gdf_unit.set_index('ID').sort_index().to_crs('epsg:4326'),gdf_poi[['code','geometry']])
    np.save(data_dir+'intermediate_results/Sequences_greedy.npy',sequences)
else:
    sequences = np.load(data_dir+'intermediate_results/Sequences_greedy.npy',allow_pickle=True).item()

In [11]:
if not model_exist:
    corpus = [TaggedDocument([str(x) for x in words], [f'd{idx}'])for idx, words in sequences.items()]
    model = gensim.models.doc2vec.Doc2Vec(dm=1, vector_size=72, dm_mean=1, window=5, dbow_words=1, min_count=1, epochs=100, seed=1,workers=1)
    model.build_vocab(corpus)
    model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs)
    model.save(data_dir+'intermediate_results/doc2vec_model')
else:
    model = gensim.models.doc2vec.Doc2Vec.load(data_dir+'intermediate_results/doc2vec_model')

In [12]:
# DataFrame 12,392 * (index, ZoneVec_1, ZoneVec_2, ...)
if not model_exist:
    df_zonevec = pd.DataFrame.from_dict({'index':sequences.keys()})
    i = 1
    for v in model.dv.vectors.T:
        df_zonevec[f"ZoneVec_{i}"] = v
        i += 1
    df_zonevec = df_zonevec.set_index("index").sort_index()
    df_zonevec.to_csv(data_dir+"intermediate_results/df_zonevec.csv",index=True)
else:
    df_zonevec = pd.read_csv(data_dir+'intermediate_results/df_zonevec.csv')

## Flow

In [13]:
def clean_df(dfs):
    df = dfs.copy()
    df["hour"] = df.period.apply(lambda x: x % 100)
    df["day"] = df.date.apply(lambda x: x % 100)
    df["time"] = (df["day"]-1)*24+df["hour"]
    df_reind = df[["o_grid", "d_grid", "time","cu_pop"]]
    return df_reind

In [14]:
if not flow_exist:
    p_od = data_dir + 'org_data/cyt_hourod2_sz_201811_250m_test84.csv'
    df_od = pd.read_csv(p_od,low_memory=False)
    p_map = data_dir + 'org_data/tid_sz_250_wgs84_0.txt'
    df_map = pd.read_csv(p_map)
    df_map['o_grid'] = df_map['TID'].astype(int)
    df_map['d_grid'] = df_map['TID'].astype(int)
    df_map['o_id'] = df_map['OID_'].astype(int)
    df_map['d_id'] = df_map['OID_'].astype(int)
    go_dict = df_map.set_index(['o_grid'])['o_id'].to_dict()
    gd_dict = df_map.set_index(['d_grid'])['d_id'].to_dict()
    df_od['o_tid'] = df_od['o_grid'].astype(int)
    df_od['d_tid'] = df_od['d_grid'].astype(int)
    df_od['o_grid'] = df_od['o_tid'].map(go_dict)
    df_od['d_grid'] = df_od['d_tid'].map(gd_dict)
    df_od = clean_df(df_od)
    df_od.to_csv(data_dir+'intermediate_results/total_flow.csv',index=False)
else:
    df_od = pd.read_csv(data_dir+'intermediate_results/total_flow.csv')

## Self-reminder

In [15]:
# 1.self target - df_area [ID, 公共管理与公共服务用地, 公用设施用地, 商业服务设施用地, 居住用地, 工业用地, 绿地广场用地, 道路与交通设施用地, 非建设用地] 33,054
# 2.self property - df_zonevec [index, ZoneVec_1, ...] 12,392 ∈ 33,054 because some regions' pois are below 10.
# 3.spatial interaction - df_od [o_grid, d_grid, cu_pop]

## Data clean

In [16]:
if not clean_exist:
    zone_idx = df_zonevec['index'].unique().tolist()
    df_od = df_od[df_od["o_grid"].isin(zone_idx)]
    df_od = df_od[df_od["d_grid"].isin(zone_idx)]
    union_idx = set(df_od.o_grid).union(set(df_od.d_grid))
    dic_idx = {x:i for x,i in zip(union_idx,range(len(union_idx)))}
    with open(data_dir+'intermediate_results/dic_map.pkl','wb') as f:
        pickle.dump(dic_idx,f)
    df_zonevec = df_zonevec[df_zonevec['index'].isin(union_idx)]
    def zone_remap(dfs,dic_idx):
        df = dfs.copy()
        df['idx'] = df['index'].map(dic_idx)
        return df
    df_zonevec = zone_remap(df_zonevec,dic_idx)
    df_zonevec.set_index('idx').sort_index().to_csv(data_dir+"final_input/df_zonevec.csv")
    df_area = df_area[df_area['ID'].isin(union_idx)]
    def area_remap(dfs,dic_idx):
        df = dfs.copy()
        df['idx'] = df["ID"].map(dic_idx)
        return df
    df_area = area_remap(df_area,dic_idx)
    df_area.set_index('idx').sort_index().to_csv(data_dir+'final_input/df_area.csv')
    gdf_unit = gdf_unit[gdf_unit["ID"].isin(union_idx)]
    def unit_remap(dfs,dic_idx):
        df = dfs.copy()
        df['idx'] = df['ID'].map(dic_idx)
        return df
    gdf_unit = unit_remap(gdf_unit,dic_idx)
    gdf_unit.set_index('idx').sort_index().to_file(data_dir+'final_input/unit.shp',encoding='utf-8')
    def od_remap(dfs,dic_idx):
        df = dfs.copy()
        df['o_id'] = df['o_grid'].map(dic_idx)
        df['d_id'] = df['d_grid'].map(dic_idx)
        return df[['o_id','d_id','cu_pop']]
    df_od = od_remap(df_od,dic_idx)
    flow_mx = np.zeros((len(union_idx),len(union_idx)),dtype=float)
    flow_len = df_od.shape[0]
    df_od = df_od.reset_index()
    for i in range(flow_len):
        o_idx = int(df_od.loc[i,'o_id'])
        d_idx = int(df_od.loc[i,'d_id'])
        cu_pop = float(df_od.loc[i,'cu_pop'])
        flow_mx[o_idx,d_idx] = flow_mx[o_idx,d_idx] + cu_pop
    flow_mx = flow_mx/30.0
    np.save(data_dir+'final_input/flow_mx.npy',flow_mx)
else:
    df_area = pd.read_csv(data_dir+'final_input/df_area.csv')
    df_zonevec = pd.read_csv(data_dir+'final_input/df_zonevec.csv')
    gdf_unit = gpd.read_file(data_dir+'final_input/unit.shp')
    flow_mx = np.load(data_dir+'final_input/flow_mx.npy')

## Distance-decay & Adjacency

In [ ]:
if not dis_exist:
    df_dis = gdf_unit.geometry.centroid.apply(lambda g: gdf_unit.geometry.centroid.distance(g))
    df_dis.to_csv(data_dir+'intermediate_results/df_dis.csv')
    dis = df_dis.values[:,1:]
    beta = 1.5
    def weight(x,beta=beta):
        return np.log((1+max_data**beta)/(1+x**beta))
    df_dis = pd.DataFrame(dis)
    max_data = df_dis.max().max()
    df_wdis = df_dis.applymap(weight)
    mx = df_wdis.values
    def normalize(matx):
        rowsum = np.array(matx.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0.
        r_mat_inv = np.diag(r_inv)
        matx = r_mat_inv.dot(matx)
        return matx
    mx = normalize(mx)
    np.save(data_dir+'final_input/wdis_normalize.npy',mx)
else:
    dis = np.load(data_dir+'final_input/wdis_normalize.npy')

In [18]:
if not adj_exist:
    ls_edge = []
    for _,row in gdf_unit.iterrows():
        neighbors = gdf_unit[~gdf_unit.geometry.disjoint(row.geometry)]['idx'].tolist()
        for i in neighbors:
            if row['idx']<i:
                a = [row['idx'],i,1]
                ls_edge.append(a)
    with open(data_dir+'final_input/ls_edge.pkl', 'wb') as f:
        pickle.dump(ls_edge, f) 
    adj = np.zeros(dis.shape)
    for i in ls_edge:
        adj[i[0],i[1]] = 1
        adj[i[1],i[0]] = 1
    adj = adj + np.eye(adj.shape[0])
    np.save(data_dir+'final_input/adj.npy',adj)
else:
    adj = np.load(data_dir+'final_input/adj.npy')

## Summary (input - interaction - output)

### Ⅰ - input (property - Doc2Vec Embedding)

In [20]:
# 12,345 * 72
# df_zonevec

### Ⅱ - interaction (flow/distance-decay/adjacency)

In [26]:
# Flow (12,345, 12,345)
# flow_mx
# Distance-decay (12,345, 12,345)
# dis
# Adjacency (12,345,12,345)
# adj

### Ⅲ - output (urban function - LU area)

In [28]:
# Land use (12,345,8)
# df_area